# Type Safety
See Lott & Phillips, 4th edition, chapter 5, page 164, "Treat objects as objects".

Strict type safety is not possible since Python uses dynamic typing. When used intentionally, this is called Monkey Typing. Testing is one application: swapping a live-data class for a test-data class at run time. 

We can approximate type safety using static type checking. Python itself does not do this, but tools like mypy an pylint do it. They check the code without running it. They cannot catch every type violation, but they can be more effective than "just running the code" to test it.



## Example: a distance function

In [1]:
# This function computes the hypotenuse of a right triangle.
from math import hypot
print('hypot(3,4)=',hypot(3,4))   # famous examle using integers
print('hypot(5,12)=',hypot(5,12))  # another famous example
print('hypot(6,10)=',hypot(6,10))   # but in general, the answer is not an integer

hypot(3,4)= 5.0
hypot(5,12)= 13.0
hypot(6,10)= 11.661903789690601


In [2]:
# Our distance function builds on the hypot function.
# Our function computes the distance between points on the 2D plane.
# It effectively draws a right triangle and computes the hypotenuse.
def distance(p1,p2):
    '''Assumption: each point has X and Y coordinates on the 2D plane.'''
    horizontal = p1[0]-p2[0]
    vertical   = p1[1]-p2[1]
    return hypot(horizontal,vertical)

In [3]:
# This client uses our function correctly.
def good_client(distance_function):
    print('Good client ... ',end='')
    p1 = (0,0)
    p2 = (3,4)
    try:
        print('distance = ',
              distance_function( p1, p2) , '(right)')  
    except Exception as ex:
        print('There was an exception!')
        print(ex)
good_client(distance)

Good client ... distance =  5.0 (right)


## Client misuse of the distance function

In [4]:
# This client uses our function incorrectly by passing 3D points.
# The two points are far from each other along the z axis.
# But our function ignores the z axis.
# The answer is very wrong but the client may never know it.
def bad_client(distance_function):
    print('Bad client ... ',end='')
    p1 = (0,0,0)
    p2 = (3,4,100)
    try:
        print('distance = ',
              distance_function( p1, p2) , '(wrong)')  
    except Exception as ex:
        print('There was an exception!')
        print(ex)
bad_client(distance)

Bad client ... distance =  5.0 (wrong)


In [5]:
# The pylint tool does not catch this problem. Try it!
# The mypy tool does not detect this problem. Try it!

## Attempts at type safety

In [6]:
# This wrapper function adds type checking at run time.
# But this approach has disadvantages.
# Having to type all this undoes some benefits of using point ojects.
# Having to run all this slows the program at run time.
# Having to write all this test code is itself error-prone.
# And there is an unintended consequence:
# since our code handles it, mypy does not flag the bad client!
def guarded_distance(p1,p2):  # wrapper
    msg = 'Both parameters must be 2D points!'
    try:
        p1x=float(p1[0])
        p2x=float(p2[0])
        p1y=float(p1[1])
        p2y=float(p2[1])
    except:
        raise Exception(msg)
    try:
        p1z=float(p1[2])
        p2z=float(p2[2])
    except:
        pass
    else:
        raise Exception(msg)
    return distance(p1,p2)  # pass-thru
good_client(guarded_distance)
bad_client(guarded_distance)

Good client ... distance =  5.0 (right)
Bad client ... There was an exception!
Both parameters must be 2D points!


In [7]:
# This code communicates our intentions to client programmers. 
# Of course Python doesn't check the type hints.
# Sadly, mypy finds no errors with the bad client.
def tuple_distance(p1:tuple[float,float],p2:tuple[float,float]):
    return distance(p1,p2)
good_client(tuple_distance)
bad_client(tuple_distance)

Good client ... distance =  5.0 (right)
Bad client ... distance =  5.0 (wrong)


In [8]:
# This is similar to the above, but uses typing.Tuple.
# This code communicates our intentions to client programmers. 
# Python still doesn't check the type hints.
# And mypy still finds no errors with the bad client.
from typing import Tuple
def typing_distance(p1:Tuple[float,float],p2:Tuple[float,float]):
    return distance(p1,p2)
good_client(typing_distance)
bad_client(typing_distance)

Good client ... distance =  5.0 (right)
Bad client ... distance =  5.0 (wrong)


In [9]:
# This version also communicates our intentions to client programmers.
# But neither Python or mypy spot the error in bad client.
Point = Tuple[float,float]
print('What is Point?', type(Point))
def type_safe_distance(p1:Point,p2:Point):
    return distance(p1,p2)
good_client(type_safe_distance)
bad_client(type_safe_distance)

What is Point? <class 'typing._GenericAlias'>
Good client ... distance =  5.0 (right)
Bad client ... distance =  5.0 (wrong)


## Use object-oriented instead!

In [11]:
class Point2D():
    def __init__(self,x:float,y:float):
        self.x = x
        self.y = y
    def __repr__(self):
        return 'Point2D('+str(self.x)+','+str(self.y)+')'
    def __getitem__(self,i):
        if i==0:
            return self.x
        if i==1:
            return self.y
        raise Exception("Index out of bounds: "+str(i))
    def distance(self,other:"Point2D"):
        horizontal = self.x-other.x
        vertical   = self.y-other.y
        return hypot(horizontal,vertical)        


In [13]:
def class_client():
    p1 = Point2D(0,0)
    p2 = Point2D(3,4)
    dist = p1.distance(p2)
    print('Points', p1, p2, 'distance',dist)
class_client()

Points Point2D(0,0) Point2D(3,4) distance 5.0


In [14]:
# Finally, we have some pretty safe code.
# Python finds no errors with this.
# Also, mypy finds no errors with this.
# The client would have to work pretty hard to break this.